In [1]:
# Log in to your W&B account
import wandb
import random
import math

In [5]:
import wandb
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier

# 1차

In [33]:

# 1. wandb 로그인 (API key를 이용)
wandb.login(key="8bab4affd59dd33bd76096b069d74f6dc0abb056")

# 2. 데이터 불러오기 및 전처리
train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')  # 필요시 불러오기

# ID 컬럼 제거 (예시로 'ID', 'id', 'RecordID', 'patient_id' 등)
for col in ['id', 'ID', 'RecordID', 'patient_id']:
    if col in train_df.columns:
        train_df = train_df.drop(col, axis=1)
    # if col in test_df.columns:
    #     test_df = test_df.drop(col, axis=1)

# 타깃과 피처 분리 (타깃 컬럼명이 '임신 성공 여부'라고 가정)
target = '임신 성공 여부'
X = train_df.drop(target, axis=1)
y = train_df[target]

# 범주형, 수치형 변수 지정
# (아래 목록은 예시이므로 실제 데이터에 맞게 수정하세요)
categorical_columns = [
    '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형',  # "특정 시술 유형"은 문자열이므로 반드시 범주형에 포함!
    '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부',
    '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
    '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
    '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인',
    '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인',
    '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
    '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
    '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유',
    '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
    '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수',
    'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처',
    '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부',
    '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부',
    'PGD 시술 여부', 'PGS 시술 여부'
]

numerical_columns = [col for col in X.columns if col not in categorical_columns]

# 전처리 파이프라인
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# 3. wandb 스윕(run)을 위한 함수 정의
def run():
    # 각 run마다 wandb 초기화 (sweep 에이전트가 호출)
    wandb.init(project="LGaimers", entity="espada105-hanseouniversity")
    config = wandb.config

    # 모델 생성: wandb.config에서 하이퍼파라미터 값을 가져옴
    model = LGBMClassifier(
        n_estimators=int(config.n_estimators),
        learning_rate=config.learning_rate,
        max_depth=int(config.max_depth),
        random_state=42
    )

    # 파이프라인 구성 (전처리 + 분류기)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    # 교차 검증 (5-fold)로 성능 평가
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc')
    mean_score = np.mean(scores)
    
    # 결과 로깅
    wandb.log({'roc_auc': mean_score})
    print(f"Mean ROC AUC: {mean_score:.4f}")
    wandb.finish()

# 4. 스윕 설정 구성
sweep_config = {
    'method': 'grid',  # 'grid', 'random', 'bayes'
    'metric': {
        'name': 'roc_auc',
        'goal': 'maximize'
    },
    'parameters': {
        'n_estimators': {
            'values': [100, 200]
        },
        'learning_rate': {
            'values': [0.01, 0.05, 0.1]
        },
        'max_depth': {
            'values': [5, 7, 10]
        }
    }
}

# 스윕 생성 (sweep_id 반환)
sweep_id = wandb.sweep(sweep_config, project="LGaimers", entity="espada105-hanseouniversity")
print("Sweep ID:", sweep_id)

# 5. 에이전트를 사용해 스윕 실행 (예: 10번의 실험)
wandb.agent(sweep_id, function=run, count=20)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\tjddl\_netrc


Create sweep with ID: d6kdsk99
Sweep URL: https://wandb.ai/espada105-hanseouniversity/LGaimers/sweeps/d6kdsk99
Sweep ID: d6kdsk99


wandb: Agent Starting Run: rpir7gr0 with config:
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 5
wandb: 	n_estimators: 100


c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\d

Mean ROC AUC: 0.7293


...

c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\d

Mean ROC AUC: 0.7388


c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


roc_auc,▁
roc_auc,0.73876


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# 2차

In [ ]:
import pandas as pd
import numpy as np
import wandb

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from category_encoders import TargetEncoder
from lightgbm import early_stopping, log_evaluation

# -----------------------------
# 1. wandb 로그인 및 스윕 설정
# -----------------------------
wandb.login(key="8bab4affd59dd33bd76096b069d74f6dc0abb056")

# 스윕 설정 (여기서는 그리드 서치를 예시로 함)
sweep_config = {
    'method': 'grid',  # 'grid', 'random', 'bayes' 중 선택
    'metric': {
        'name': 'final_validation_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'n_estimators': {'values': [1000, 1500]},
        'learning_rate': {'values': [0.01, 0.005]},
        'max_depth': {'values': [10, 12]},
        # 나머지 고정 파라미터는 'value'로 지정합니다.
        'stopping_rounds': {'value': 100},
        'test_size': {'value': 0.2},
        'random_state': {'value': 42}
    }
}

sweep_id = wandb.sweep(sweep_config, project="LGaimers", entity="espada105-hanseouniversity")
print("Sweep ID:", sweep_id)

# -----------------------------
# 2. run() 함수 정의: 하이퍼파라미터에 따른 모델 학습 및 평가
# -----------------------------
def run():
    # 각 run마다 wandb 초기화 및 config 로드
    wandb.init(project="LGaimers", entity="espada105-hanseouniversity")
    config = wandb.config

    # --- 데이터 불러오기 및 전처리 ---
    # 학습 데이터 불러오기 (ID 컬럼 제거)
    train_df = pd.read_csv('train.csv')
    train_df = train_df.drop(columns=['ID'])
    
    # 타깃과 피처 분리 (타깃 컬럼명이 '임신 성공 여부'라고 가정)
    X = train_df.drop('임신 성공 여부', axis=1)
    y = train_df['임신 성공 여부']
    
    # 범주형 변수 목록 (실제 데이터에 맞게 수정)
    categorical_columns = [
        "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부",
        "배란 유도 유형", "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부",
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인",
        "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 남성 요인",
        "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제",
        "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인",
        "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태", "배아 생성 주요 이유", "총 시술 횟수",
        "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수",
        "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처",
        "난자 기증자 나이", "정자 기증자 나이", "동결 배아 사용 여부", "신선 배아 사용 여부",
        "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부"
    ]
    numerical_columns = [col for col in X.columns if col not in categorical_columns]
    
    # TargetEncoder를 이용하여 범주형 변수 인코딩
    encoder = TargetEncoder(cols=categorical_columns)
    X_encoded = encoder.fit_transform(X, y)
    
    # train/validation 세트 분할 (config에 설정된 test_size와 random_state 사용)
    X_train, X_val, y_train, y_val = train_test_split(
        X_encoded, y, test_size=config.test_size, random_state=config.random_state
    )
    
    # --- 초기 모델 학습 및 피처 선택 ---
    base_model = lgb.LGBMClassifier(
        n_estimators=config.n_estimators,
        learning_rate=config.learning_rate,
        max_depth=config.max_depth,
        random_state=config.random_state
    )
    callbacks = [early_stopping(stopping_rounds=config.stopping_rounds, verbose=True)]
    
    base_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
    
    # SelectFromModel을 이용해 피처 선택 (중간값(threshold='median') 기준)
    selector = SelectFromModel(base_model, prefit=True, threshold='median')
    X_train_selected = selector.transform(X_train)
    X_val_selected = selector.transform(X_val)
    
    # --- 최종 모델 학습 ---
    final_model = lgb.LGBMClassifier(
        n_estimators=config.n_estimators,
        learning_rate=config.learning_rate,
        max_depth=config.max_depth,
        random_state=config.random_state
    )
    callbacks_final = [
        early_stopping(stopping_rounds=config.stopping_rounds, verbose=True),
        log_evaluation(10)
    ]
    final_model.fit(X_train_selected, y_train, eval_set=[(X_val_selected, y_val)], callbacks=callbacks_final)
    
    # 검증 데이터에 대한 예측 및 평가
    from sklearn.metrics import roc_auc_score

    # 최종 모델 예측 확률 (양성 클래스에 대한 확률)
    y_pred_proba_final = final_model.predict_proba(X_val_selected)[:, 1]
    roc_auc = roc_auc_score(y_val, y_pred_proba_final)
    print(f"최종 모델 ROC AUC: {roc_auc:.4f}")
    wandb.log({"final_validation_roc_auc": roc_auc})

    wandb.finish()

# -----------------------------
# 3. wandb 에이전트를 이용해 스윕 실행
# -----------------------------
wandb.agent(sweep_id, function=run, count=10)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\tjddl\_netrc


Create sweep with ID: dxgv2s1t
Sweep URL: https://wandb.ai/espada105-hanseouniversity/LGaimers/sweeps/dxgv2s1t
Sweep ID: dxgv2s1t


wandb: Agent Starting Run: pm2e545k with config:
wandb: 	colsample_bytree: 0.6
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 10
wandb: 	min_child_weight: 1
wandb: 	n_estimators: 1000
wandb: 	random_state: 42
wandb: 	reg_alpha: 0
wandb: 	reg_lambda: 0
wandb: 	stopping_rounds: 100
wandb: 	subsample: 0.6
wandb: 	test_size: 0.2


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[949]	valid_0's binary_logloss: 0.485906


c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Training until validation scores don't improve for 100 rounds
[10]	valid_0's binary_logloss: 0.557659
[20]	valid_0's binary_logloss: 0.548339
[30]	valid_0's binary_logloss: 0.540488
[40]	valid_0's binary_logloss: 0.533832
[50]	valid_0's binary_logloss: 0.528117
[60]	valid_0's binary_logloss: 0.523202
[70]	valid_0's binary_logloss: 0.518968
[80]	valid_0's binary_logloss: 0.515291
[90]	valid_0's binary_logloss: 0.512077
[100]	valid_0's binary_logloss: 0.509274
[110]	valid_0's binary_logloss: 0.506829
[120]	valid_0's binary_logloss: 0.504677
[130]	valid_0's binary_logloss: 0.502781
[140]	valid_0's binary_logloss: 0.501087
[150]	valid_0's binary_logloss: 0.499581
[160]	valid_0's binary_logloss: 0.498259
[170]	valid_0's binary_logloss: 0.49708
[180]	valid_0's binary_logloss: 0.496036
[190]	valid_0's binary_logloss: 0.495096
[200]	valid_0's binary_logloss: 0.494264
[210]	valid_0's binary_logloss: 0.493512
[220]	valid_0's binary_logloss: 0.492837
[230]	valid_0's binary_logloss: 0.492232
[240]

c:\Users\tjddl\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


최종 모델 ROC AUC: 0.73926345


final_validation_roc_auc,▁
final_validation_roc_auc,0.73926
